In [4]:
import requests

import pandas as pd

In [16]:
def get_nearest(coords, node_id, filename):
    lat, lon = coords
    response = requests.get(f"http://127.0.0.1:6000/nearest/v1/foot/{lon},{lat}")
    response.raise_for_status()

    columns = [
        'osm_node_id',
        'osm_node_lat',
        'osm_node_lon',
        'node_id',
        'filename',
    ]
    df = pd.DataFrame(columns=columns)

    data = response.json()
    if 'waypoints' in data:
        for waypoint in data['waypoints']:
            lon, lat = waypoint['location']
            for osm_node_id in waypoint['nodes']:
                df = pd.concat([
                    df, 
                    pd.DataFrame.from_records([{
                        'node_id': node_id,
                        'osm_node_id': osm_node_id,
                        'osm_node_lat': lat,
                        'osm_node_lon': lon,
                        'filename': filename
                    }])
                ], ignore_index=True)

    return df

def query_osrm(df):
    results = []

    for _, row in df.iterrows():
        nearest_df = get_nearest(
            (row['lat'], row['lon']), 
            int(row['node_id']), 
            row['filename'],
        )
        results.append(nearest_df)

    return pd.concat(results).reset_index(drop=True)   

In [17]:
df = pd.read_csv('../nodes/gpx-nodes.csv')

osrm_df = query_osrm(df)

osrm_df.to_csv(
    '../nodes/osm-nodes.csv',
    index=False,
)